<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> **Finanzas Cuantitativas | Otoño 2021**. 

**Nombres:** 
- Juan Pablo Ruíz.
- Rubén Hernández.
- Bryan Juárez.

**Fecha:** 27 de mayo del 2021.
    
<img style="float: right; margin: 30px 15px 15px 15px;" src="https://www.monash.edu/__data/assets/image/0004/1017517/iStock-611890326-OK3.jpg" width="300" height="500" />

**Expedientes** : 
- 721093.
- 717710.
- 722176.

**Profesor:** José Mario Zárate.  

## Tarea 1.

### Ejercicio 1.1

Use the Csv files that contains 2000 records of the sales of similar products in the (each columns represents a year).

Develop a function in python where you can evaluate a project usingIRR and NPV.

The inputs is the flows by year (csv file), the output is a data frame containing NPV and IRR. 

Add lines to the code to answer the 4 question of the activity:
- 1°- What's the probability that the IRR is over the Risk free rate?
- 2°- What's the probability that the IRR is over the $35\%$?
- 3°- What's the probability that the project value is over $\$2\text{M}$?
- 4°- What's the probability that the IRR is between $10\%$ and $20\%$?

In [1]:
# Descargamos libreria financiera de python.
!pip install numpy-financial

In [2]:
# Librerías a utilizar.
import numpy as np
import pandas as pd
import numpy_financial as npf

In [3]:
# Comenzamos importando nuestros datos.
data_oil = pd.read_csv('Data_OilCompany.csv')
data_oil.head(10)

,Year 1,Year 2,Year 3,Year 4,Year 5
0,"203,726.00","279,969.00","465,303.00","200,445.00","96,319.00"
1,"263,845.00","272,439.00","2,192.00","199,901.00",-
2,"27,726.00","334,861.00","82,265.00","199,954.00","64,958.00"
3,"293,801.00","210,117.00","275,087.00","199,934.00",-
4,"297,319.00","242,569.00","464,097.00","200,598.00","141,285.00"
5,"36,012.00","296,235.00","484,943.00","199,842.00",-
6,"78,138.00","308,996.00","491,647.00","199,218.00","127,187.00"
7,"61,767.00","269,747.00","23,989.00","199,771.00",-
8,"346,503.00","172,903.00","41,218.00","200,827.00","80,270.00"
9,"149,681.00","205,417.00","441,224.00","199,658.00",-


**Data Cleansing**.

A simple vista se puede observar que el formato específicado no ayuda a la manipulación de los datos, por lo tanto trabajaremos transformandolos de manera que podamos manejar valores númericos.

In [4]:
# Definimos una función para reemplazar caracteres.
def replace_text(x, to_replace, replacement):
    try:
        x = x.replace(to_replace, replacement)
    except:
        pass
    return x

In [5]:
# Primero iteramos para eliminar los espacios en las celdas de los valores y remplazamos el signo '-' por un 0.
data_oil_values = pd.DataFrame([[(replace_text(data_oil.iloc[i,j], ',', '').strip()) 
                                  for i in range(len(data_oil))] 
                                  for j in range(len(data_oil.columns))][0:5]).replace('-', '0').T

# Iteramos para volver numéricos los valores.
data_oil_values = pd.DataFrame([[int(float(data_oil_values.iloc[i,j]))
                                 for i in range(len(data_oil_values))]
                                 for j in range(len(data_oil_values.columns))])

# Visualizamos.
data_oil_values.T.head(10)

,0,1,2,3,4
0,203726,279969,465303,200445,96319
1,263845,272439,2192,199901,0
2,27726,334861,82265,199954,64958
3,293801,210117,275087,199934,0
4,297319,242569,464097,200598,141285
5,36012,296235,484943,199842,0
6,78138,308996,491647,199218,127187
7,61767,269747,23989,199771,0
8,346503,172903,41218,200827,80270
9,149681,205417,441224,199658,0


In [6]:
# Por cuestión de estética reestructuramos la información en un nuevo data frame.
data_oil_final = pd.DataFrame(np.asmatrix(data_oil_values.T),
                              columns = ['Year 1', 'Year 2', 'Year 3', 'Year 4', 'Year 5'])

# Visualizamos.
data_oil_final.index.name = 'Scenarios'
data_oil_final.head(10)

,Year 1,Year 2,Year 3,Year 4,Year 5
Scenarios,,,,,
0,203726,279969,465303,200445,96319
1,263845,272439,2192,199901,0
2,27726,334861,82265,199954,64958
3,293801,210117,275087,199934,0
4,297319,242569,464097,200598,141285
5,36012,296235,484943,199842,0
6,78138,308996,491647,199218,127187
7,61767,269747,23989,199771,0
8,346503,172903,41218,200827,80270


**Function Definition**.

Ya con los datos en un formato numérico y manipulable, lo que haremos ahora será definir la función con la cuál evaluaremos un proyecto mediante las métricas del **Valor Presente Neto** y la **Tasa Interna de Retorno**.

In [7]:
# Definimos la función a trabajar.
def get_project_valuation(data: 'DataFrame donde se almacenan los posibles valores de producción.',
                          invest: 'Inversión inicial.',
                          price: 'Precio de venta por unidad.',
                          cost: 'Costo fijo por unidad.',
                          fixed: 'Costo de mantenimiento anual.',
                          rf: 'Tasa libre de riesgo.'):
    
    '''La intención de está función es evaluar un proyecto a partir de las métricas del NPV y IRR.'''
    
    ### Definimos un data frame donde almacenaremos los flujos.
    # Flujos de inversión inicial.
    investment_flow = pd.DataFrame([-abs(invest)] * len(data_oil_final))
    
    # Flujos correspondientes a las cantidades producidas.
    flows = pd.DataFrame([[data.T.iloc[i,j] * (price-cost) - fixed 
                           for i in range(len(data.columns))]
                           for j in range(len(data))])
    
    # Concatenamos los flujos.
    final_flows = pd.merge(investment_flow, flows, right_index=True, left_index=True)
    
    ### Calculamos el valor presente neto, NPV.
    npv_vector = [(npf.npv(rf, final_flows.iloc[i,:])) for i in range(len(final_flows))]
    
    ### Calculamos la tasa interna de retorno, IRR.
    irr_vector = [round(npf.irr(final_flows.iloc[i,:]), 2) for i in range(len(final_flows))]
    
    ### Estructuramos información final en un data frame.
    results = pd.DataFrame(columns=['NPV', 'IRR'])
    results.index.name = 'Scenarios'
    results['NPV'] = npv_vector
    results['IRR'] = irr_vector
    
    ### Retornamos el data frame creado.
    return results

In [8]:
### Probamos la función creada.
# Definimos parámetros de entrada.
invest = 1800000
price = 5
cost = 0.8
fixed = 40000
rf = 0.03

# Probamos el output de la función.
project_valuation = get_project_valuation(data_oil_final, invest, price, cost, fixed, rf)
project_valuation.head(10)

,NPV,IRR
Scenarios,,
0,2.841295e+06,0.51
1,9.256289e+05,0.27
2,7.532422e+05,0.17
3,1.850074e+06,0.44
4,3.233719e+06,0.59
5,1.946086e+06,0.35
6,2.652614e+06,0.42
7,1.742579e+05,0.07
8,1.312898e+06,0.34


**Questions**.

Del **output** mostrado se puede verificar que los resultados obtenidos han sido satisfactorios, con esto en cuenta ahora lo que haremos será comenzar a dar respuesta a las preguntas planteadas desde un enfoque _frecuentista_ de la probabilidad.

1°- What's the probability that the IRR is over the Risk free rate?

In [9]:
# Respondemos.
prob_question1 = (sum([1 if i > rf else 0 for i in project_valuation.IRR])) / len(project_valuation.IRR)
print(f'La probabilidad de que la TIR supere la tasa libre de riesgo es de: {(prob_question1)*100}%')

La probabilidad de que la TIR supere la tasa libre de riesgo es de: 98.7%


2°- What's the probability that the IRR is over the $35\%$?

In [10]:
# Respondemos.
prob_question2 = (sum([1 if i > 0.35 else 0 for i in project_valuation.IRR])) / len(project_valuation.IRR)
print(f'La probabilidad de que la TIR supere un 35% es de: {(prob_question2)*100}%')

La probabilidad de que la TIR supere un 35% es de: 52.1%


3°- What's the probability that the project value is over $\$2\text{M}$?

In [11]:
# Respondemos.
prob_question3 = (sum([1 if i > 2000000 else 0 for i in project_valuation.NPV])) / len(project_valuation.NPV)
print(f'La probabilidad de que el VPN supere los 2 millones de dólares es de: {round((prob_question3)*100, 4)}%')

La probabilidad de que el VPN supere los 2 millones de dólares es de: 40.4%


4°- What's the probability that the IRR is between $10\%$ and $20\%$?

In [12]:
# Respondemos.
prob_question4 = (sum([1 if 0.1<=i<=0.2 else 0 for i in project_valuation.IRR])) / len(project_valuation.IRR)
print(f'La probabilidad de que la TIR este entre el 10% y el 20% es de: {round((prob_question4)*100, 4)}%')

La probabilidad de que la TIR este entre el 10% y el 20% es de: 13.1%


### Ejercicio 1.2

Select one of the options from slide 4 and propose adding theuncertainty factor in one of the variables.
- 1°- Explain clearly your conditions.
- 2°- Draw the decision tree and expected value.
- 3°- Explain under what conditions your decisions changes.

**Escribir solución**.